In [24]:
import mysql.connector
import sys
import os
import pandas as pd

In [4]:
credentials_path = r'E:\StuDY\FIT5120\db_keys'
sys.path.append(credentials_path)
from db_keys import mysql_host, mysql_user, mysql_password, mysql_database

In [43]:
import pymysql

# Connect to your MySQL database

mydb = mysql.connector.connect(
    host=mysql_host,
    user=mysql_user,
    password=mysql_password,
    database=mysql_database
)

mycursor = mydb.cursor()

In [39]:
df = pd.read_excel('2020_median_income.xlsx')
# Reshape the DataFrame into the format required for the 'individual' table
df_females = df[['STATE', 'Age Range', 'Females']].rename(columns={'Females': 'med_salary'})
df_females['gender'] = 'F'
df_females['med_salary'] = df_females['med_salary'].astype(int)
df_males = df[['STATE', 'Age Range', 'Males']].rename(columns={'Males': 'med_salary'})
df_males['gender'] = 'M'
df_males['med_salary'] = df_males['med_salary'].astype(int)
df_individual = pd.concat([df_females, df_males], ignore_index=True)
df_individual['med_salary'] = df_individual['med_salary'].astype(int)

In [26]:
# Insert gender data into the `gender` table

gender_data = [
    ('F', 'Female'),
    ('M', 'Male'),
]

sql = "INSERT INTO gender (gender, gender_desc) VALUES (%s, %s)"
val = gender_data
mycursor.executemany(sql, val)

mydb.commit()

In [27]:
# Insert state data into the `state` table
state_data = [
    ('ACT', 'Australian Capital Territory'),
    ('NSW', 'New South Wales'),
    ('NT', 'Northern Territory'),
    ('QLD', 'Queensland'),
    ('SA', 'South Australia'),
    ('TAS', 'Tasmania'),
    ('VIC', 'Victoria'),
    ('WA', 'Western Australia')
]

sql = "INSERT INTO state (state_abr, state_name) VALUES (%s, %s)"
val = state_data
mycursor.executemany(sql, val)

mydb.commit()

In [44]:
try:

    # Insert individual data
    for index, row in df_individual.iterrows():
        sql = f"""
        INSERT INTO individual (state_abr, age_range, gender, med_salary)
        VALUES ('{row['STATE']}', '{row['Age Range']}', '{row['gender']}', {row['med_salary']});
        """
        mycursor.execute(sql)

    mydb.commit()

finally:
    mydb.close()

: 